In [ ]:
import os
import glob
from pathlib import Path
import pandas as pd
import torch
from fastprogress import progress_bar
import numpy as np
import warnings
from collections import defaultdict
from collections import Counter

In [ ]:
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(42)

# 1. Prelim

In [ ]:
ROOT = Path.cwd().parent
INPUT_ROOT = ROOT / "input"
RAW_DATA = INPUT_ROOT / "birdclef-2021"

if(os.path.isdir(INPUT_ROOT / "birdclef-2021" / "testsoundscapes")):
    TEST_AUDIO_DIR = INPUT_ROOT / "birdclef-2021" / "testsoundscapes"
    test = pd.read_csv(INPUT_ROOT / "birdclef-2021" / "test.csv")
else:
    TEST_AUDIO_DIR = INPUT_ROOT / "birdclef-2021" / "train_soundscapes/"
    test = pd.read_csv(INPUT_ROOT / "birdclef-2021" / "train_soundscape_labels.csv")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
BIRD_CODE = {
    'acafly': 0, 'acowoo': 1, 'aldfly': 2, 'ameavo': 3, 'amecro': 4, 'amegfi': 5,
    'amekes': 6, 'amepip': 7, 'amered': 8, 'amerob': 9, 'amewig': 10,
    'amtspa': 11, 'andsol1': 12, 'annhum': 13, 'astfly': 14, 'azaspi1': 15,
    'babwar': 16, 'baleag': 17, 'balori': 18, 'banana': 19, 'banswa': 20,
    'banwre1': 21, 'barant1': 22, 'barswa': 23, 'batpig1': 24, 'bawswa1': 25,
    'bawwar': 26, 'baywre1': 27, 'bbwduc': 28, 'bcnher': 29, 'belkin1': 30,
    'belvir': 31, 'bewwre': 32, 'bkbmag1': 33, 'bkbplo': 34, 'bkbwar': 35,
    'bkcchi': 36, 'bkhgro': 37, 'bkmtou1': 38, 'bknsti': 39, 'blbgra1': 40,
    'blbthr1': 41, 'blcjay1': 42, 'blctan1': 43, 'blhpar1': 44, 'blkpho': 45,
    'blsspa1': 46, 'blugrb1': 47, 'blujay': 48, 'bncfly': 49, 'bnhcow': 50,
    'bobfly1': 51, 'bongul': 52, 'botgra': 53, 'brbmot1': 54, 'brbsol1': 55,
    'brcvir1': 56, 'brebla': 57, 'brncre': 58, 'brnjay': 59, 'brnthr': 60,
    'brratt1': 61, 'brwhaw': 62, 'brwpar1': 63, 'btbwar': 64, 'btnwar': 65,
    'btywar': 66, 'bucmot2': 67, 'buggna': 68, 'bugtan': 69, 'buhvir': 70,
    'bulori': 71, 'burwar1': 72, 'bushti': 73, 'butsal1': 74, 'buwtea': 75,
    'cacgoo1': 76, 'cacwre': 77, 'calqua': 78, 'caltow': 79, 'cangoo': 80,
    'canwar': 81, 'carchi': 82, 'carwre': 83, 'casfin': 84, 'caskin': 85,
    'caster1': 86, 'casvir': 87, 'categr': 88, 'ccbfin': 89, 'cedwax': 90,
    'chbant1': 91, 'chbchi': 92, 'chbwre1': 93, 'chcant2': 94, 'chispa': 95,
    'chswar': 96, 'cinfly2': 97, 'clanut': 98, 'clcrob': 99, 'cliswa': 100,
    'cobtan1': 101, 'cocwoo1': 102, 'cogdov': 103, 'colcha1': 104, 'coltro1': 105,
    'comgol': 106, 'comgra': 107, 'comloo': 108, 'commer': 109, 'compau': 110,
    'compot1': 111, 'comrav': 112, 'comyel': 113, 'coohaw': 114, 'cotfly1': 115,
    'cowscj1': 116, 'cregua1': 117, 'creoro1': 118, 'crfpar': 119, 'cubthr': 120,
    'daejun': 121, 'dowwoo': 122, 'ducfly': 123, 'dusfly': 124, 'easblu': 125,
    'easkin': 126, 'easmea': 127, 'easpho': 128, 'eastow': 129, 'eawpew': 130,
    'eletro': 131, 'eucdov': 132, 'eursta': 133, 'fepowl': 134, 'fiespa': 135,
    'flrtan1': 136, 'foxspa': 137, 'gadwal': 138, 'gamqua': 139, 'gartro1': 140,
    'gbbgul': 141, 'gbwwre1': 142, 'gcrwar': 143, 'gilwoo': 144, 'gnttow': 145,
    'gnwtea': 146, 'gocfly1': 147, 'gockin': 148, 'gocspa': 149, 'goftyr1': 150,
    'gohque1': 151, 'goowoo1': 152, 'grasal1': 153, 'grbani': 154, 'grbher3': 155,
    'grcfly': 156, 'greegr': 157, 'grekis': 158, 'grepew': 159, 'grethr1': 160,
    'gretin1': 161, 'greyel': 162, 'grhcha1': 163, 'grhowl': 164, 'grnher': 165,
    'grnjay': 166, 'grtgra': 167, 'grycat': 168, 'gryhaw2': 169, 'gwfgoo': 170,
    'haiwoo': 171, 'heptan': 172, 'hergul': 173, 'herthr': 174, 'herwar': 175,
    'higmot1': 176, 'hofwoo1': 177, 'houfin': 178, 'houspa': 179, 'houwre': 180,
    'hutvir': 181, 'incdov': 182, 'indbun': 183, 'kebtou1': 184, 'killde': 185,
    'labwoo': 186, 'larspa': 187, 'laufal1': 188, 'laugul': 189, 'lazbun': 190,
    'leafly': 191, 'leasan': 192, 'lesgol': 193, 'lesgre1': 194, 'lesvio1': 195,
    'linspa': 196, 'linwoo1': 197, 'littin1': 198, 'lobdow': 199, 'lobgna5': 200,
    'logshr': 201, 'lotduc': 202, 'lotman1': 203, 'lucwar': 204, 'macwar': 205,
    'magwar': 206, 'mallar3': 207, 'marwre': 208, 'mastro1': 209, 'meapar': 210,
    'melbla1': 211, 'monoro1': 212, 'mouchi': 213, 'moudov': 214, 'mouela1': 215,
    'mouqua': 216, 'mouwar': 217, 'mutswa': 218, 'naswar': 219, 'norcar': 220,
    'norfli': 221, 'normoc': 222, 'norpar': 223, 'norsho': 224, 'norwat': 225,
    'nrwswa': 226, 'nutwoo': 227, 'oaktit': 228, 'obnthr1': 229, 'ocbfly1': 230,
    'oliwoo1': 231, 'olsfly': 232, 'orbeup1': 233, 'orbspa1': 234, 'orcpar': 235,
    'orcwar': 236, 'orfpar': 237, 'osprey': 238, 'ovenbi1': 239, 'pabspi1': 240,
    'paltan1': 241, 'palwar': 242, 'pasfly': 243, 'pavpig2': 244, 'phivir': 245,
    'pibgre': 246, 'pilwoo': 247, 'pinsis': 248, 'pirfly1': 249, 'plawre1': 250,
    'plaxen1': 251, 'plsvir': 252, 'plupig2': 253, 'prowar': 254, 'purfin': 255,
    'purgal2': 256, 'putfru1': 257, 'pygnut': 258, 'rawwre1': 259, 'rcatan1': 260,
    'rebnut': 261, 'rebsap': 262, 'rebwoo': 263, 'redcro': 264, 'reevir1': 265,
    'rehbar1': 266, 'relpar': 267, 'reshaw': 268, 'rethaw': 269, 'rewbla': 270,
    'ribgul': 271, 'rinkin1': 272, 'roahaw': 273, 'robgro': 274, 'rocpig': 275,
    'rotbec': 276, 'royter1': 277, 'rthhum': 278, 'rtlhum': 279, 'ruboro1': 280,
    'rubpep1': 281, 'rubrob': 282, 'rubwre1': 283, 'ruckin': 284, 'rucspa1': 285,
    'rucwar': 286, 'rucwar1': 287, 'rudpig': 288, 'rudtur': 289, 'rufhum': 290,
    'rugdov': 291, 'rumfly1': 292, 'runwre1': 293, 'rutjac1': 294, 'saffin': 295,
    'sancra': 296, 'sander': 297, 'savspa': 298, 'saypho': 299, 'scamac1': 300,
    'scatan': 301, 'scbwre1': 302, 'scptyr1': 303, 'scrtan1': 304, 'semplo': 305,
    'shicow': 306, 'sibtan2': 307, 'sinwre1': 308, 'sltred': 309, 'smbani': 310,
    'snogoo': 311, 'sobtyr1': 312, 'socfly1': 313, 'solsan': 314, 'sonspa': 315,
    'soulap1': 316, 'sposan': 317, 'spotow': 318, 'spvear1': 319, 'squcuc1': 320,
    'stbori': 321, 'stejay': 322, 'sthant1': 323, 'sthwoo1': 324, 'strcuc1': 325,
    'strfly1': 326, 'strsal1': 327, 'stvhum2': 328, 'subfly': 329, 'sumtan': 330,
    'swaspa': 331, 'swathr': 332, 'tenwar': 333, 'thbeup1': 334, 'thbkin': 335,
    'thswar1': 336, 'towsol': 337, 'treswa': 338, 'trogna1': 339, 'trokin': 340,
    'tromoc': 341, 'tropar': 342, 'tropew1': 343, 'tuftit': 344, 'tunswa': 345,
    'veery': 346, 'verdin': 347, 'vigswa': 348, 'warvir': 349, 'wbwwre1': 350,
    'webwoo1': 351, 'wegspa1': 352, 'wesant1': 353, 'wesblu': 354, 'weskin': 355,
    'wesmea': 356, 'westan': 357, 'wewpew': 358, 'whbman1': 359, 'whbnut': 360,
    'whcpar': 361, 'whcsee1': 362, 'whcspa': 363, 'whevir': 364, 'whfpar1': 365,
    'whimbr': 366, 'whiwre1': 367, 'whtdov': 368, 'whtspa': 369, 'whwbec1': 370,
    'whwdov': 371, 'wilfly': 372, 'willet1': 373, 'wilsni1': 374, 'wiltur': 375,
    'wlswar': 376, 'wooduc': 377, 'woothr': 378, 'wrenti': 379, 'y00475': 380,
    'yebcha': 381, 'yebela1': 382, 'yebfly': 383, 'yebori1': 384, 'yebsap': 385,
    'yebsee1': 386, 'yefgra1': 387, 'yegvir': 388, 'yehbla': 389, 'yehcar1': 390,
    'yelgro': 391, 'yelwar': 392, 'yeofly1': 393, 'yerwar': 394, 'yeteup1': 395,
    'yetvir': 396
}

INV_BIRD_CODE = {v: k for k, v in BIRD_CODE.items()}

# 2. Models

## 2.1 Audio Utils

In [ ]:
'''
ISC License
Copyright (c) 2013--2017, librosa development team.

Permission to use, copy, modify, and/or distribute this software for any purpose with or without fee is hereby granted, provided that the above copyright notice and this permission notice appear in all copies.

THE SOFTWARE IS PROVIDED "AS IS" AND THE AUTHOR DISCLAIMS ALL WARRANTIES WITH REGARD TO THIS SOFTWARE INCLUDING ALL IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS. IN NO EVENT SHALL THE AUTHOR BE LIABLE FOR ANY SPECIAL, DIRECT, INDIRECT, OR CONSEQUENTIAL DAMAGES OR ANY DAMAGES WHATSOEVER RESULTING FROM LOSS OF USE, DATA OR PROFITS, WHETHER IN AN ACTION OF CONTRACT, NEGLIGENCE OR OTHER TORTIOUS ACTION, ARISING OUT OF OR IN CONNECTION WITH THE USE OR PERFORMANCE OF THIS SOFTWARE.
'''

import torch.nn as nn
import numpy as np
import torch
import librosa
import torch.nn.functional as F
class DFTBase(nn.Module):
    def __init__(self):
        """Base class for DFT and IDFT matrix"""
        super(DFTBase, self).__init__()

    def dft_matrix(self, n):
        (x, y) = np.meshgrid(np.arange(n), np.arange(n))
        omega = np.exp(-2 * np.pi * 1j / n)
        W = np.power(omega, x * y)
        return W

    def idft_matrix(self, n):
        (x, y) = np.meshgrid(np.arange(n), np.arange(n))
        omega = np.exp(2 * np.pi * 1j / n)
        W = np.power(omega, x * y)
        return W
    
    
class STFT(DFTBase):
    def __init__(self, n_fft=2048, hop_length=None, win_length=None, 
        window='hann', center=True, pad_mode='reflect', freeze_parameters=True):
        """Implementation of STFT with Conv1d. The function has the same output 
        of librosa.core.stft
        """
        super(STFT, self).__init__()

        assert pad_mode in ['constant', 'reflect']

        self.n_fft = n_fft
        self.center = center
        self.pad_mode = pad_mode

        # By default, use the entire frame
        if win_length is None:
            win_length = n_fft

        # Set the default hop, if it's not already specified
        if hop_length is None:
            hop_length = int(win_length // 4)

        fft_window = librosa.filters.get_window(window, win_length, fftbins=True)

        # Pad the window out to n_fft size
        fft_window = librosa.util.pad_center(fft_window, n_fft)

        # DFT & IDFT matrix
        self.W = self.dft_matrix(n_fft)

        out_channels = n_fft // 2 + 1

        self.conv_real = nn.Conv1d(in_channels=1, out_channels=out_channels, 
            kernel_size=n_fft, stride=hop_length, padding=0, dilation=1, 
            groups=1, bias=False)

        self.conv_imag = nn.Conv1d(in_channels=1, out_channels=out_channels, 
            kernel_size=n_fft, stride=hop_length, padding=0, dilation=1, 
            groups=1, bias=False)

        self.conv_real.weight.data = torch.Tensor(
            np.real(self.W[:, 0 : out_channels] * fft_window[:, None]).T)[:, None, :]
        # (n_fft // 2 + 1, 1, n_fft)

        self.conv_imag.weight.data = torch.Tensor(
            np.imag(self.W[:, 0 : out_channels] * fft_window[:, None]).T)[:, None, :]
        # (n_fft // 2 + 1, 1, n_fft)

        if freeze_parameters:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, input):
        """input: (batch_size, data_length)
        Returns:
          real: (batch_size, n_fft // 2 + 1, time_steps)
          imag: (batch_size, n_fft // 2 + 1, time_steps)
        """

        x = input[:, None, :]   # (batch_size, channels_num, data_length)

        if self.center:
            x = F.pad(x, pad=(self.n_fft // 2, self.n_fft // 2), mode=self.pad_mode)

        real = self.conv_real(x)
        imag = self.conv_imag(x)
        # (batch_size, n_fft // 2 + 1, time_steps)

        real = real[:, None, :, :].transpose(2, 3)
        imag = imag[:, None, :, :].transpose(2, 3)
        # (batch_size, 1, time_steps, n_fft // 2 + 1)

        return real, imag
    
    
class Spectrogram(nn.Module):
    def __init__(self, n_fft=2048, hop_length=None, win_length=None, 
        window='hann', center=True, pad_mode='reflect', power=2.0, 
        freeze_parameters=True):
        """Calculate spectrogram using pytorch. The STFT is implemented with 
        Conv1d. The function has the same output of librosa.core.stft
        """
        super(Spectrogram, self).__init__()

        self.power = power

        self.stft = STFT(n_fft=n_fft, hop_length=hop_length, 
            win_length=win_length, window=window, center=center, 
            pad_mode=pad_mode, freeze_parameters=True)

    def forward(self, input):
        """input: (batch_size, 1, time_steps, n_fft // 2 + 1)
        Returns:
          spectrogram: (batch_size, 1, time_steps, n_fft // 2 + 1)
        """

        (real, imag) = self.stft.forward(input)
        # (batch_size, n_fft // 2 + 1, time_steps)

        spectrogram = real ** 2 + imag ** 2

        if self.power == 2.0:
            pass
        else:
            spectrogram = spectrogram ** (power / 2.0)

        return spectrogram

    
class LogmelFilterBank(nn.Module):
    def __init__(self, sr=32000, n_fft=2048, n_mels=64, fmin=50, fmax=14000, is_log=True, 
        ref=1.0, amin=1e-10, top_db=80.0, freeze_parameters=True):
        """Calculate logmel spectrogram using pytorch. The mel filter bank is 
        the pytorch implementation of as librosa.filters.mel 
        """
        super(LogmelFilterBank, self).__init__()

        self.is_log = is_log
        self.ref = ref
        self.amin = amin
        self.top_db = top_db

        self.melW = librosa.filters.mel(sr=sr, n_fft=n_fft, n_mels=n_mels,
            fmin=fmin, fmax=fmax).T
        # (n_fft // 2 + 1, mel_bins)

        self.melW = nn.Parameter(torch.Tensor(self.melW))

        if freeze_parameters:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, input):
        """input: (batch_size, channels, time_steps)
        
        Output: (batch_size, time_steps, mel_bins)
        """

        # Mel spectrogram
        mel_spectrogram = torch.matmul(input, self.melW)

        # Logmel spectrogram
        if self.is_log:
            output = self.power_to_db(mel_spectrogram)
        else:
            output = mel_spectrogram

        return output


    def power_to_db(self, input):
        """Power to db, this function is the pytorch implementation of 
        librosa.core.power_to_lb
        """
        ref_value = self.ref
        log_spec = 10.0 * torch.log10(torch.clamp(input, min=self.amin, max=np.inf))
        log_spec -= 10.0 * np.log10(np.maximum(self.amin, ref_value))

        if self.top_db is not None:
            if self.top_db < 0:
                raise ParameterError('top_db must be non-negative')
            log_spec = torch.clamp(log_spec, min=log_spec.max().item() - self.top_db, max=np.inf)

        return log_spec


class DropStripes(nn.Module):
    def __init__(self, dim, drop_width, stripes_num):
        """Drop stripes. 
        Args:
          dim: int, dimension along which to drop
          drop_width: int, maximum width of stripes to drop
          stripes_num: int, how many stripes to drop
        """
        super(DropStripes, self).__init__()

        assert dim in [2, 3]    # dim 2: time; dim 3: frequency

        self.dim = dim
        self.drop_width = drop_width
        self.stripes_num = stripes_num

    def forward(self, input):
        """input: (batch_size, channels, time_steps, freq_bins)"""

        assert input.ndimension() == 4

        if self.training is False:
            return input

        else:
            batch_size = input.shape[0]
            total_width = input.shape[self.dim]

            for n in range(batch_size):
                self.transform_slice(input[n], total_width)

            return input


    def transform_slice(self, e, total_width):
        """e: (channels, time_steps, freq_bins)"""

        for _ in range(self.stripes_num):
            distance = torch.randint(low=0, high=self.drop_width, size=(1,))[0]
            bgn = torch.randint(low=0, high=total_width - distance, size=(1,))[0]

            if self.dim == 2:
                e[:, bgn : bgn + distance, :] = 0
            elif self.dim == 3:
                e[:, :, bgn : bgn + distance] = 0


class SpecAugmentation(nn.Module):
    def __init__(self, time_drop_width, time_stripes_num, freq_drop_width, 
        freq_stripes_num):
        """Spec augmetation. 
        [ref] Park, D.S., Chan, W., Zhang, Y., Chiu, C.C., Zoph, B., Cubuk, E.D. 
        and Le, Q.V., 2019. Specaugment: A simple data augmentation method 
        for automatic speech recognition. arXiv preprint arXiv:1904.08779.
        Args:
          time_drop_width: int
          time_stripes_num: int
          freq_drop_width: int
          freq_stripes_num: int
        """

        super(SpecAugmentation, self).__init__()

        self.time_dropper = DropStripes(dim=2, drop_width=time_drop_width, 
            stripes_num=time_stripes_num)

        self.freq_dropper = DropStripes(dim=3, drop_width=freq_drop_width, 
            stripes_num=freq_stripes_num)

    def forward(self, input):
        x = self.time_dropper(input)
        x = self.freq_dropper(x)
        return x

## 2.2 PANN Models

In [ ]:
'''
The MIT License
  
Copyright (c) 2018-2020 Qiuqiang Kong

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.
'''


import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)


def init_bn(bn):
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.0)


def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.

    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    """Pad framewise_output to the same length as input frames. The pad value
    is the same as the value of the last frame.
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1:, :].repeat(
        1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output


class ConvBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()

        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
            bias=False)

        self.conv2 = nn.Conv2d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
            bias=False)

        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()

    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

    def forward(self, input, pool_size=(2, 2), pool_type='avg'):

        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')

        return x


class AttBlock(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear",
                 temperature=1.0):
        super().__init__()

        self.activation = activation
        self.temperature = temperature
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.bn_att = nn.BatchNorm1d(out_features)
        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)
        init_bn(self.bn_att)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)
        
class PANNsDense121Att(nn.Module):
    def __init__(self, sample_rate: int, window_size: int, hop_size: int,
                 mel_bins: int, fmin: int, fmax: int, classes_num: int, apply_aug: bool, top_db=None):
        super().__init__()
        
        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        self.interpolate_ratio = 32  # Downsampled ratio
        self.apply_aug = apply_aug

        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(
            n_fft=window_size,
            hop_length=hop_size,
            win_length=window_size,
            window=window,
            center=center,
            pad_mode=pad_mode,
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(
            sr=sample_rate,
            n_fft=window_size,
            n_mels=mel_bins,
            fmin=fmin,
            fmax=fmax,
            ref=ref,
            amin=amin,
            top_db=top_db,
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(
            time_drop_width=64,
            time_stripes_num=2,
            freq_drop_width=8,
            freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(mel_bins)

        self.fc1 = nn.Linear(1024, 1024, bias=True)
        self.att_block = AttBlock(1024, classes_num, activation='sigmoid')


        self.densenet_features = models.densenet121(pretrained=False).features

        self.init_weight()

    def init_weight(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        
    def cnn_feature_extractor(self, x):
        x = self.densenet_features(x)
        return x
    
    def preprocess(self, input_x, mixup_lambda=None):

        x = self.spectrogram_extractor(input_x)  # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor(x)  # (batch_size, 1, time_steps, mel_bins)

        frames_num = x.shape[2]

        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)

        if self.apply_aug:
            x = self.spec_augmenter(x)

        return x, frames_num
        

    def forward(self, input_data):
        input_x, mixup_lambda = input_data
        """
        Input: (batch_size, data_length)"""
        b, c, s = input_x.shape
        input_x = input_x.reshape(b*c, s)
        x, frames_num = self.preprocess(input_x, mixup_lambda=mixup_lambda)
        if mixup_lambda is not None:
            b = (b*c)//2
            c = 1
        # Output shape (batch size, channels, time, frequency)
        x = x.expand(x.shape[0], 3, x.shape[2], x.shape[3])
        x = self.cnn_feature_extractor(x)
        
        # Aggregate in frequency axis
        x = torch.mean(x, dim=3)

        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2

        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)

        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        # Get framewise output
        framewise_output = interpolate(segmentwise_output,
                                       self.interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)
        frame_shape =  framewise_output.shape
        clip_shape = clipwise_output.shape
        output_dict = {
            'framewise_output': framewise_output.reshape(b, c, frame_shape[1],frame_shape[2]),
            'clipwise_output': clipwise_output.reshape(b, c, clip_shape[1]),
        }

        return output_dict

## 2.3 Model Utils

In [ ]:
def get_model(ModelClass: object, config: dict, weights_path: str):
    model = ModelClass(**config)
    checkpoint = torch.load(weights_path, map_location='cpu')
    model.load_state_dict(checkpoint["model"])
    model.to(device)
    model.eval()
    return model

# 3. Model Parameters

In [ ]:
list_of_models = [
    {
        "model_class": PANNsDense121Att,
        "config": {
            "sample_rate": 32000,
            "window_size": 1024,
            "hop_size": 320,
            "mel_bins": 64,
            "fmin": 50,
            "fmax": 14000,
            "classes_num": len(BIRD_CODE),
            "apply_aug": True,
            "top_db": None
        },
        "weights_path": "../input/birdclef-2021-training/saved_models/example_config/example_config_checkpoint_2_score=0.0107.pt",
        "clip_threshold": 0.1, # 0.3,
        "threshold": 0.1 #0.3
    }
]
PERIOD = 30
SR = 32000
# how many models are neccessary for a successful detection
vote_lim = 1 # 4
TTA = 10

In [ ]:
for lm in list_of_models:
    lm["model"] = get_model(lm["model_class"], lm["config"], lm["weights_path"])

# 4. Predictions

In [ ]:
def prediction_for_clip(test_df: pd.DataFrame,
                        clip: np.ndarray, 
                        model,
                        threshold,
                       clip_threshold):

    audios = []
    y = clip.astype(np.float32)
    len_y = len(y)
    start = 0
    end = PERIOD * SR
    while True:
        y_batch = y[start:end].astype(np.float32)
        if len(y_batch) != PERIOD * SR:
            y_pad = np.zeros(PERIOD * SR, dtype=np.float32)
            y_pad[:len(y_batch)] = y_batch
            audios.append(y_pad)
            break
        start = end
        end += PERIOD * SR
        audios.append(y_batch)
        
    array = np.asarray(audios)
    tensors = torch.from_numpy(array)
    
    model.eval()
    estimated_event_list = []
    global_time = 0.0
    site = test_df["site"].values[0]
    audio_id = test_df["audio_id"].values[0]
    for image in tensors:
        image = image.unsqueeze(0).unsqueeze(0)
        image = image.expand(image.shape[0], TTA, image.shape[2])
        image = image.to(device)
        
        with torch.no_grad():
            prediction = model((image, None))
            framewise_outputs = prediction["framewise_output"].detach(
                ).cpu().numpy()[0].mean(axis=0)
            clipwise_outputs = prediction["clipwise_output"].detach(
                ).cpu().numpy()[0].mean(axis=0)
                
        thresholded = framewise_outputs >= threshold
        
        clip_thresholded = clipwise_outputs >= clip_threshold
        clip_indices = np.argwhere(clip_thresholded).reshape(-1)
        clip_codes = []
        for ci in clip_indices:
            clip_codes.append(INV_BIRD_CODE[ci])
            
        for target_idx in range(thresholded.shape[1]):
            if thresholded[:, target_idx].mean() == 0:
                pass
            else:
                detected = np.argwhere(thresholded[:, target_idx]).reshape(-1)
                head_idx = 0
                tail_idx = 0
                while True:
                    if (tail_idx + 1 == len(detected)) or (
                            detected[tail_idx + 1] - 
                            detected[tail_idx] != 1):
                        onset = 0.01 * detected[
                            head_idx] + global_time
                        offset = 0.01 * detected[
                            tail_idx] + global_time
                        onset_idx = detected[head_idx]
                        offset_idx = detected[tail_idx]
                        max_confidence = framewise_outputs[
                            onset_idx:offset_idx, target_idx].max()
                        mean_confidence = framewise_outputs[
                            onset_idx:offset_idx, target_idx].mean()
                        if INV_BIRD_CODE[target_idx] in clip_codes:
                            estimated_event = {
                                "site": site,
                                "audio_id": audio_id,
                                "primary_label": INV_BIRD_CODE[target_idx],
                                "clip_codes": clip_codes,
                                "onset": onset,
                                "offset": offset,
                                "max_confidence": max_confidence,
                                "mean_confidence": mean_confidence
                            }
                            estimated_event_list.append(estimated_event)
                        head_idx = tail_idx + 1
                        tail_idx = tail_idx + 1
                        if head_idx >= len(detected):
                            break
                    else:
                        tail_idx += 1
        global_time += PERIOD
        
    prediction_df = pd.DataFrame(estimated_event_list)
    return prediction_df

def prediction(test_df: pd.DataFrame,
               test_audio: Path,
               list_of_model_details):
    test_df['audio_site_id'] = test_df['audio_id'].map(str) + '_' + test_df['site'].map(str)
    unique_audio_site_id = test_df.audio_site_id.unique()

    warnings.filterwarnings("ignore")
    prediction_dfs_dict = defaultdict(list)
    for audio_site_id in progress_bar(unique_audio_site_id):
        search_pattern = os.path.join(test_audio, audio_site_id + '_*.ogg')
        audio_id = int(audio_site_id.split('_')[0])
        audio_filenames = glob.glob(search_pattern)
        if(len(audio_filenames) == 0):
            continue
        
        audio_filename = audio_filenames[0]
        clip, _ = librosa.load(audio_filename, sr=SR, mono=True, res_type="kaiser_fast")
        
        test_df_for_audio_id = test_df.query(f"audio_id == '{audio_id}'").reset_index(drop=True)
        for i, model_details in enumerate(list_of_model_details):
            prediction_df = prediction_for_clip(test_df_for_audio_id,
                                                clip=clip,
                                                model=model_details["model"],
                                                threshold=model_details["threshold"],
                                                clip_threshold=model_details["clip_threshold"]
                                               )
            prediction_dfs_dict[i].append(prediction_df)
    list_of_prediction_df = []
    for key, prediction_dfs in prediction_dfs_dict.items():
        prediction_df = pd.concat(prediction_dfs, axis=0, sort=False).reset_index(drop=True)
        list_of_prediction_df.append(prediction_df)
    return list_of_prediction_df


In [ ]:
list_of_prediction_df = prediction(test_df=test,
                           test_audio=TEST_AUDIO_DIR,
                           list_of_model_details=list_of_models)

# 5. Post Process

In [ ]:
def get_post_post_process_predictions(prediction_df):
    labels = {}
    for audio_id, sub_df in progress_bar(prediction_df.groupby("audio_id")):
        events = sub_df[["primary_label", "onset", "offset", "max_confidence", "site"]].values
        n_events = len(events)

        site = events[0][4]
        for i in range(n_events):
            event = events[i][0]
            onset = events[i][1]
            offset = events[i][2]
            
            start_section = int((onset // 5) * 5) + 5
            end_section = int((offset // 5) * 5) + 5
            cur_section = start_section

            row_id = f"{audio_id}_{site}_{start_section}" #  f"{site}_{audio_id}_{start_section}"
            if labels.get(row_id) is not None:
                labels[row_id].add(event)
            else:
                labels[row_id] = set()
                labels[row_id].add(event)

            while cur_section != end_section:
                cur_section += 5
                row_id = f"{audio_id}_{site}_{cur_section}" # f"{site}_{audio_id}_{cur_section}"
                if labels.get(row_id) is not None:
                    labels[row_id].add(event)
                else:
                    labels[row_id] = set()
                    labels[row_id].add(event)


    for key in labels:
        labels[key] = " ".join(sorted(list(labels[key])))


    row_ids = list(labels.keys())
    birds = list(labels.values())
    post_processed = pd.DataFrame({
        "row_id": row_ids,
        "birds": birds
    })
    return post_processed

In [ ]:
all_row_id = test[["row_id"]]
list_of_submissions = []
for prediction_df in list_of_prediction_df:
    post_processed = get_post_post_process_predictions(prediction_df)
    submission = post_processed.fillna("nocall")
    submission = submission.set_index('row_id')
    list_of_submissions.append(submission)

list_of_submissions

In [ ]:
list_all_of_row_ids = []
for sub_x in list_of_submissions:
    list_all_of_row_ids+= list(sub_x.index.values)
list_all_of_row_ids = list(set(list_all_of_row_ids))

# 6. Ensemble

In [ ]:
final_submission = []
for row_id in list_all_of_row_ids:
    birds = []
    for sub in list_of_submissions:
        if row_id in sub.index:
            birds.extend(sub.loc[row_id].birds.split(" "))
    birds = [x for x in birds if "nocall" != x and "" != x]
    count_birds = Counter(birds)
    final_birds = []
    for key, value in count_birds.items():
        if value >= vote_lim:
            final_birds.append(key)
    if len(final_birds)>0:
        row_data = {
            "row_id": row_id,
            "birds": " ".join(sorted(final_birds))
        }
    else:
        row_data = {
            "row_id": row_id,
            "birds": "nocall"
        }
    final_submission.append(row_data)

site_3_data = defaultdict(list)
for row in final_submission:
    if "site_3" in row["row_id"]:
        final_row_id = "_".join(row["row_id"].split("_")[0:-1])
        birds = row["birds"].split(" ")
        birds = [x for x in birds if "nocall" != x and "" != x]
        site_3_data[final_row_id].extend(birds)
        
for key, value in site_3_data.items():
    count_birds = Counter(value)
    final_birds = []
    for k, v in count_birds.items():
        if v >= vote_lim:
            final_birds.append(k)
    if len(final_birds)>0:
        row_data = {
            "row_id": key,
            "birds": " ".join(sorted(final_birds))
        }
    else:
        row_data = {
            "row_id": key,
            "birds": "nocall"
        }
    final_submission.append(row_data)

submission = pd.DataFrame(final_submission)
submission = all_row_id.merge(submission, on="row_id", how="left")
submission = submission.fillna("nocall")

In [ ]:
submission.head(500)

In [ ]:
submission.to_csv("submission.csv", index=False)